#5

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%reset-f
import numpy as np
import pandas as pd
import random

#normalising the instances
def normal(X_tr,X_val,X_te):
  mean=X_tr.mean(axis=1)
  std=X_tr.std(axis=1)
  for i in range(X_tr.shape[0]):
    X_tr[i:i+1,:]=(X_tr[i:i+1,:]-mean[i])/std[i]
    X_val[i:i+1,:]=(X_val[i:i+1,:]-mean[i])/std[i]
    X_te[i:i+1,:]=(X_te[i:i+1,:]-mean[i])/std[i]
  return X_tr,X_val,X_te

#splitting instances into training,testing and validation
def split_inst(inst,train_ratio,val_ratio):
  X=inst[:inst.shape[0]-1,:]
  Y=inst[inst.shape[0]-1:,:]
  X_tr=X[:,:int(train_ratio*X.shape[1])]
  Y_tr=Y[:,:int(train_ratio*Y.shape[1])]
  X_val=X[:,int(train_ratio*X.shape[1]):int((train_ratio+val_ratio)*X.shape[1])]
  Y_val=Y[:,int(train_ratio*Y.shape[1]):int((train_ratio+val_ratio)*Y.shape[1])]
  X_te=X[:,int((train_ratio+val_ratio)*X.shape[1]):]
  Y_te=Y[:,int((train_ratio+val_ratio)*Y.shape[1]):]
  return X_tr,X_val,X_te,Y_tr,Y_val,Y_te

#Running all functions invloved in preprocessing of the instances
def preprocess(inst):
  inst=df.values
  inst=Randomize(inst)
  inst=inst.T
  X_tr,X_val,X_te,Y_tr,Y_val,Y_te=split_inst(inst,0.7,0.1)
  X_tr,X_val,X_te=normal(X_tr,X_val,X_te)
  Y_tr=hot_encoding(Y_tr,3)
  Y_val=hot_encoding(Y_val,3)
  Y_te=hot_encoding(Y_te,3)
  return X_tr,X_val,X_te,Y_tr,Y_val,Y_te

#randomly shuffling the instances
def Randomize(inst):
  np.random.shuffle(inst)
  return inst

def activation(Z,k):
  if k== 1:
    A=(np.exp(Z)-np.exp(-Z))/(np.exp(Z)+np.exp(-Z))
  if k==2:
    A=1/(1+np.exp(-Z))
  return A

def cost(a,b):
  return (1/a.shape[1])*np.sum((a-b)**2)

def return_max(Y_hat):
  for i in range(Y_hat.shape[1]):
    if Y_hat[0][i]>Y_hat[1][i] and Y_hat[0][i]>Y_hat[2][i]:
      Y_hat[0][i]=1
      Y_hat[1][i]=0
      Y_hat[2][i]=0
    elif Y_hat[1][i]>Y_hat[0][i] and Y_hat[1][i]>Y_hat[2][i]:
      Y_hat[0][i]=0
      Y_hat[1][i]=1
      Y_hat[2][i]=0
    else:
      Y_hat[0][i]=0
      Y_hat[1][i]=0
      Y_hat[2][i]=1
  return Y_hat


def accuracy(Y_hat,Y):
  a=np.abs(Y_hat-Y)
  num_mistakes=(np.sum(a))/2
  acc=100*(Y.shape[1]-num_mistakes)/Y.shape[1]
  return acc

def hot_encoding(Y,n_Y):
  Y_new=np.zeros((n_Y,Y.shape[1]))
  for i in range(Y.shape[1]):
    Y_new[int(Y[0,i])-1][i]=1
  return Y_new

  

#STACKED AUTOENCODER

In [8]:
def pretraining(input,n_h):
  W1=np.random.randn(n_h,input.shape[0])
  W2=np.random.randn(input.shape[0],n_h)
  b1=np.zeros((n_h,1))
  b2=np.zeros((input.shape[0],1))
  m=input.shape[1]
  for i in range(1000):
    #forward propagation
    Z1=np.dot(W1,input)+b1
    A=activation(Z1,1)
    Z2=np.dot(W2,A)+b2
    output=activation(Z2,1)
    #backpropagation
    dZ2=(output-input)*(1-output**2)
    dW2=(1/m)*np.dot(dZ2,A.T)
    db2=(1/m)*np.sum(dZ2,axis=1,keepdims=True)
    dA=np.dot(W2.T,dZ2)
    dZ1=dA*(1-A**2)
    dW1=(1/m)*np.dot(dZ1,input.T)
    db1=(1/m)*np.sum(dZ1,axis=1,keepdims=True)
    #gradient descent
    W1=W1-0.5*dW1
    W2=W2-0.5*dW2
    b1=b1-0.5*db1
    b2=b2-0.5*db2
    #cost
  return W1,b1,A

def forward_propagation(X,parameters):
  Z1=np.dot(parameters["W1"],X)+parameters["b1"]
  A1=activation(Z1,1)
  Z2=np.dot(parameters["W2"],A1)+parameters["b2"]
  A2=activation(Z2,1)
  Z3=np.dot(parameters["W3"],A2)+parameters["b3"]
  A3=activation(Z3,1)
  Z4=np.dot(parameters["W4"],A3)+parameters["b4"]
  Y_hat= activation(Z4,2)
  nodes={"Z1":Z1,
         "A1":A1,
         "Z2":Z2,
         "A2":A2,
         "Z3":Z3,
         "A3":A3,
         "Z4":Z4,
         "Y_hat":Y_hat}
  return nodes


#calculating backward propagation for a single iteration and returning the value of the gradient of each neuron
def back_propagation(X,Y,parameters,nodes):
  m=X.shape[1]
  dZ4=(nodes["Y_hat"]-Y)*nodes["Y_hat"]*(1-nodes["Y_hat"]**2)
  dW4=(1/m)*np.dot(dZ4,nodes["A3"].T)
  db4=(1/m)*np.sum(dZ4,axis=1,keepdims=True)
  dA3=np.dot(parameters["W4"].T,dZ4)
  dZ3=dA3*(1-nodes["A3"]**2)
  dW3=(1/m)*np.dot(dZ3,nodes["A2"].T)
  db3=(1/m)*np.sum(dZ3,axis=1,keepdims=True)
  dA2=np.dot(parameters["W3"].T,dZ3)
  dZ2=dA2*(1-nodes["A2"]**2)
  dW2=(1/m)*np.dot(dZ2,nodes["A1"].T)
  db2=(1/m)*np.sum(dZ2,axis=1,keepdims=True)
  dA1=np.dot(parameters["W2"].T,dZ2)
  dZ1=dA1*(1-nodes["A1"]**2)
  dW1=(1/m)*np.dot(dZ1,X.T)
  db1=(1/m)*np.sum(dZ1,axis=1,keepdims=True)
  grads={"dW1":dW1,
         "db1":db1,
         "dW2":dW2,
         "db2":db2,
         "dW3":dW3,
         "db3":db3,
         "dW4":dW4,
         "db4":db4}
  return grads

#updating weights and biases 
def gradient_descent(parameters,grads,learning_rate):
  parameters["W1"]=parameters["W1"]-learning_rate*grads["dW1"]
  parameters["b1"]=parameters["b1"]-learning_rate*grads["db1"]
  parameters["W2"]=parameters["W2"]-learning_rate*grads["dW2"]
  parameters["b2"]=parameters["b2"]-learning_rate*grads["db2"]
  parameters["W3"]=parameters["W3"]-learning_rate*grads["dW3"]
  parameters["b3"]=parameters["b3"]-learning_rate*grads["db3"]
  parameters["W4"]=parameters["W4"]-learning_rate*grads["dW4"]
  parameters["b4"]=parameters["b4"]-learning_rate*grads["db4"]
  return parameters


def finetuning(X,parameters,Y,learning_rate):
  for i in range(1000):
    nodes=forward_propagation(X,parameters)
    grads=back_propagation(X,Y,parameters,nodes)
    parameters=gradient_descent(parameters,grads,learning_rate)
    #if i%50==0:
      #print("Cost after iteration", i, ":", cost(nodes["Y_hat"],Y))
  Y_hat=return_max(nodes["Y_hat"])
  #print("training accuracy  :", accuracy(Y_hat,Y), "%") 
  return parameters,nodes


def model_predict(X,Y,parameters):
  nodes=forward_propagation(X,parameters)
  Y_hat=nodes["Y_hat"]
  Y_hat=return_max(Y_hat)
  return accuracy(Y_hat,Y)
  

##MAIN

In [9]:
df=pd.read_excel("/content/drive/MyDrive/ML/data5.xlsx")

inst=df.values
np.random.seed(0)
X_tr,X_val,X_te,Y_tr,Y_val,Y_te=preprocess(inst)
W1,b1,A1=pretraining(X_tr,5)
W2,b2,A2=pretraining(A1,4)
W3,b3,A3=pretraining(A2,3)
W4=np.random.randn(3,3)
b4=np.zeros((3,1))
parameters={"W1":W1, "b1":b1,"W2":W2,"b2":b2,"W3":W3,"b3":b3,"W4":W4,"b4":b4}
max=0
alpha=np.arange(0.1,1.1,0.1)
acc=np.zeros(10)
for i in range(10):
  parameters,nodes=finetuning(X_tr,parameters,Y_tr,alpha[i])
  acc[i]=model_predict(X_val,Y_val,parameters)
  if acc[i]>max:
    max=acc[i]
    imax=i
    maxparameters=parameters
print("validation accuracy is", acc[imax], "%")
acc=model_predict(X_te,Y_te,maxparameters)
print("testing accuracy is", acc, "%") 

validation accuracy is 95.23809523809524 %
testing accuracy is 95.23809523809524 %
